In [1]:
import torch
import sys
sys.path.append("../src")
sys.path.append("../src/models")
from train import getModel, prepare_dataset
from experiments import experiments
import argparse
import tqdm
import numpy as np
from datasets.ConcatDataset import ConcatDataset
import pandas as pd
import geopandas as gpd
import sklearn.metrics

In [2]:
def evaluate(model, dataloader):

    probaslist = list()
    targetslist = list()
    idslist = list()
    for iteration, data in tqdm.tqdm(enumerate(dataloader)):
        inputs, targets, ids = data
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            targets = targets.cuda()

        logprobabilities, deltas, pts, budget = model.forward(inputs.transpose(1, 2))
        probaslist.append(logprobabilities.exp().cpu().detach().numpy())
        targetslist.append(targets.cpu().detach().numpy())
        idslist.append(ids.numpy())

    ids = np.hstack(idslist)
    probas = np.vstack(probaslist)
    targets = np.vstack(targetslist)[:,0]

    return ids, targets, probas

def save_csv(ids, targets, probas, outpath):
    index = ["id","label","prediction"] + ["class_"+str(idx) for idx in range(probas.shape[1])]
    df = pd.DataFrame([ids,targets,probas.argmax(1)]+[probas[:,idx] for idx in range(probas.shape[1])], index=index).T.set_index("id")
    df.index = df.index.astype(int)
    df.label = df.label.astype(int)
    df.prediction = df.prediction.astype(int)
    print("saving "+outpath)
    df.to_csv(outpath,float_format="%.4f")
    
def build_probas_df(probas, targets, ids):

    probas_df = pd.DataFrame(probas, columns=["prob_"+str(cl) for cl in np.arange(probas.shape[1])], index=ids)
    onehot = np.eye(probas.shape[1]).astype(bool)
    probas_df["pred"] = probas.argsort()[:,-1]
    probas_df["pred"].astype(int)
    assert (probas.argsort()[:,-1] == probas.argmax(1)).all()
    probas_df["pred2nd"] = probas.argsort()[:,-2]

    probas_df["prob2nd"] = probas[onehot[probas_df["pred2nd"].values]]
    probas_df["prob2nd"].astype(int)
    probas_df["maxprob"] = probas[onehot[probas_df["pred"].values]]
    probas_df["maxprob"].astype(int)

    probas_df["GRPGRPSTM"] = targets
    probas_df["GRPGRPSTM"].astype(int)
    probas_df["correct_prediction"] = probas_df["GRPGRPSTM"] == probas_df["pred"]

    probas_df["score_correct"] = probas[onehot[probas_df["GRPGRPSTM"].values]]
    
    return probas_df

holl = gpd.read_file("/data/BavarianCrops/shp/HOLL_2018_MT_pilot.shp").set_index("ID")
krum = gpd.read_file("/data/BavarianCrops/shp/KRUM_2018_MT_pilot.shp").set_index("ID")
nowa = gpd.read_file("/data/BavarianCrops/shp/NOWA_2018_MT_pilot.shp").set_index("ID")

In [3]:
experiment="tumgaf_gaf_transformer" # "tumgaf_gafall_rnn", , "tumgaf_gafall_transformer"  
for experiment in ["tumgaf_gafall_msresnet"]:
    
    args = argparse.Namespace(experiment=experiment, seed=0, batchsize=256, workers=0)
    args = experiments(args)

    traindataloader, testdataloader = prepare_dataset(args)

    dataset = ConcatDataset([traindataloader.dataset, testdataloader.dataset])
    dataloader = torch.utils.data.DataLoader(dataset=dataset,batch_size=args.batchsize, num_workers=args.workers)

    args.nclasses = traindataloader.dataset.nclasses
    classname = traindataloader.dataset.classname
    klassenname = traindataloader.dataset.klassenname
    args.seqlength = traindataloader.dataset.sequencelength
    #args.seqlength = args.samplet
    args.input_dims = traindataloader.dataset.ndims

    model = getModel(args)
    model.load(f"/data/gaf/runsv3/{experiment}/model.pth")

    ids_train, targets_train, probas_train = evaluate(model, traindataloader)
    #save_csv(ids_train, targets_train, probas_train, f"/tmp/{experiment}_train.csv")
    df_train = build_probas_df(probas_train, targets_train, ids_train)
    #shp = shp.join(df_train, on="ID")

    ids_test, targets_test, probas_test = evaluate(model, testdataloader)
    df_test = build_probas_df(probas_test, targets_test, ids_test)
    
    df = pd.concat([df_train,df_test])
    
    #shp = gpd.read_file("/data/BavarianCrops/shp/HOLL_2018_MT_pilot.shp").set_index("ID")
    shp = pd.concat([holl,krum,nowa])
    shp = shp.join(df, on="ID")
    
    print(f"saving /tmp/gaf/{experiment}.shp")
    shp.to_file(f"/tmp/gaf/{experiment}.shp",driver = 'ESRI Shapefile')
    
    print(sklearn.metrics.classification_report(df.GRPGRPSTM, df.pred),
          file=open(f"/tmp/gaf/{experiment}_report_all.txt",'w'))
    print(sklearn.metrics.classification_report(df_train.GRPGRPSTM, df_train.pred),
          file=open(f"/tmp/gaf/{experiment}_report_train.txt",'w'))
    print(sklearn.metrics.classification_report(df_test.GRPGRPSTM, df_test.pred),
          file=open(f"/tmp/gaf/{experiment}_report_test.txt",'w'))
    #save_csv(ids_train, targets_train, probas_train, f"/tmp/{experiment}_test.csv")

ValueError: Wrong experiment name tumgaf_gafall_msresnet!

In [ ]:
model

In [ ]:
!cat /tmp/test.txt

In [ ]:
traindataloader.dataset.classes

In [ ]:
model

In [ ]:
!ls /data/gaf/runsv3/tumgaf_gaf_msresnet

In [ ]:
log = pd.read_csv("/data/gaf/runsv3/tumgaf_gaf_msresnet/log.csv")
log.loc[log["mode"]=="test"].sort_values(by="kappa", ascending=False).iloc[0].epoch

In [ ]:
probas_df

In [ ]:
print("\n".join(traindataloader.dataset.classname))